In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Model Preproccessing

In [ ]:
import numpy as np 
import pandas as pd 
import os


In [ ]:
import glob
import time
import pandas as pd

from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/dataset/sqli.csv",encoding='utf-16')

In [ ]:
df

In [ ]:
df['Label'].value_counts()

In [ ]:
df.count()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( min_df=2, max_df=0.7, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(df['Sentence'].values.astype('U')).toarray()

In [ ]:
transformed_posts=pd.DataFrame(posts)

In [ ]:
df=pd.concat([df,transformed_posts],axis=1)

In [ ]:
df

# Model data split

In [ ]:
X=df[df.columns[2:]]

In [ ]:
y=df['Label']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X.head()

# Machine Learning Models

## functions

In [ ]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [ ]:
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [ ]:
def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [ ]:
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    fscore=f1score_function(true_positive, true_negative, false_positive, false_negative)
    return (accuracy,
            precision,
           recall,
            fscore)

#### Hybrid LSTM

In [ ]:
!pip uninstall keras
!pip install keras==2.4.0

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [ ]:
#%%time
#LSTM
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Flatten
from keras.layers import Dense, LSTM, Convolution1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Flatten
from keras.layers import Dense, LSTM, Convolution1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
input_dim = X_train.shape[1]  # Number of features

num_classes=2
model=Sequential()
model.add(Embedding(input_dim,50,mask_zero=True))

#model.add(Convolution1D(filters=128, kernel_size=3, padding='valid', activation='relu'))
#model.add(Convolution1D(filters=256, kernel_size=4, padding='valid', activation='relu'))
model.add(Convolution1D(filters=512, kernel_size=5, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(4))

model.add(LSTM(20,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(20,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Flatten())
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

classifier_nn = model.fit(X_train,y_train,
                    epochs=2,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=300)

pred=model.predict(X_test)
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

accuracy_score(y_test,pred_lstm)

In [ ]:
"""
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Flatten
from keras.layers import Dense, LSTM, Convolution1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional

QA_EMBED_SIZE = 64
DROPOUT_RATE = 0.3

model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(LSTM(QA_EMBED_SIZE, return_sequences=True, dropout=DROPOUT_RATE, recurrent_dropout=DROPOUT_RATE)))
model.add(Convolution1D(filters=128, kernel_size=3, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(4))
model.add(Flatten())

model.add(Dense(QA_EMBED_SIZE))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation("sigmoid"))

model.summary()
"""

https://www.kaggle.com/syedsaqlainhussain/sql-injection-dataset?select=sqliv2.csv

## Classification

#### Logistic Regression

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred=clf.predict(X_test)
accuracy_score(y_test, y_pred)

#### CNN

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Flatten
from keras.layers import Dense, LSTM, Convolution1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Flatten
from keras.layers import Dense, LSTM, Convolution1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

In [ ]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))


In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

In [ ]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=15)


In [ ]:
pred=model.predict(X_test)

In [ ]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [ ]:
accuracy_score(y_test,pred)

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred)
print(" For CNN \n Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, pred)

In [ ]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test, pred)

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test, pred)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, pred)

#### Gaussian Naive Bayes

In [ ]:
%%time
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(X_train, y_train)

pred_gnb = gnb.predict(X_test)

#### Support Vector Machine

In [ ]:
%%time
# SVM
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)
pred_svm=clf.predict(X_test)

In [ ]:
%%time
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
lsc = make_pipeline(StandardScaler(),LinearSVC(random_state=0, tol=1e-5))
lsc = lsc.fit(X_train, y_train)
pred_lsc = lsc.predict(X_test)

In [ ]:
%%time
from sklearn.svm import NuSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
nsc = make_pipeline(StandardScaler(), NuSVC())
nsc = nsc.fit(X_train, y_train)
pred_nsc = nsc.predict(X_test)

#### KNN

In [ ]:
%%time
# KNN
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

pred_knn = neigh.predict(X_test)

#### KMeans Clustering

In [ ]:
%%time
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(X_train)

pred_knn = kmeans.predict(X_test)

#### SGD Classifier

In [ ]:
%%time
from sklearn.linear_model import SGDClassifier
sgd= SGDClassifier(loss="hinge", penalty="l2", max_iter=100)
sgd.fit(X_train, y_train)

pred_sgd = sgd.predict(X_test)

#### Decision Tree

In [ ]:
%%time
# DT
from sklearn import tree

dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train, y_train)

pred_dt = dt.predict(X_test)

#### Random Forest

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10)
rf = rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)

#### AdaBoost

In [ ]:
%%time
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(n_estimators=100, random_state=0)
ab = ab.fit(X_train, y_train)
pred_ab = ab.predict(X_test)

#### Gradient Boost

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=0)
gb = gb.fit(X_train, y_train)
pred_gb = gb.predict(X_test)

#### LDA

In [ ]:
%%time
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda = lda.fit(X_train, y_train)
pred_lda = lda.predict(X_test)

#### QDA

In [ ]:
%%time
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
qda = qda.fit(X_train, y_train)
pred_qda = qda.predict(X_test)

#### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0).fit(X_train, y_train)
pred_lr = lr.predict(X_test)

#### MLP

In [ ]:
%%time
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
mlp = mlp.fit(X_train, y_train)
pred_mlp = mlp.predict(X_test)

#### XGBoost

In [ ]:
!pip install xgboost

In [ ]:
%%time
import xgboost as xgb
xgc = xgb.XGBClassifier()
'''xgc=xgb.XGBClassifier(n_estimators=2800,
    min_child_weight=0.1,
    learning_rate=0.002,
    max_depth=2,
    subsample=0.47,
    colsample_bytree=0.35,
    gamma=0.4,
    reg_lambda=0.4,
    random_state=42,
    n_jobs=-1,)'''
xgc = xgc.fit(X_train, y_train);
pred_xgc = xgc.predict(X_test)

#### Catboost

In [ ]:
!pip install catboost

In [ ]:
%%time
import catboost as cb
cat = cb.CatBoostClassifier(depth = 9, reg_lambda=0.1,
                         learning_rate = 0.09, 
                         iterations = 500)
cat= cat.fit(X_train, y_train, verbose=False, early_stopping_rounds=50);

pred_cat = cat.predict(X_test)

#### LightGBM

In [ ]:
import lightgbm as lgb
lgc = lgb.LGBMClassifier(max_depth = 7,
                         lambda_l1 = 0.1,
                         lambda_l2 = 0.01,
                         learning_rate = 0.01, 
                         n_estimators = 500, reg_alpha = 1.1, colsample_bytree = 0.9, subsample = 0.9,
                         n_jobs = 5)
lgc.fit(X_train, y_train, verbose=False,eval_metric='accuracy');
pred_lgc = lgc.predict(X_test)

# Results for table

In [ ]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall





def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy = accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision = precision_function(true_positive, false_positive)
    recall = recall_function(true_positive, false_negative)
    
    return (accuracy,precision,recall)

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,y_pred)
print(" For KNN Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,y_pred)
print(" For MLP \n Accuracy : {0} \n Precision : {1} \n Recall : {2} ".format(accuracy,precision,recall)

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred_gnb)
print(" For Naive Bayes Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred_svm)
print(" For SVM Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred_knn)
print(" For KNN Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred_dt)
print(" For Decision Tree Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred_ab)
print(" For ADaboost: {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred_xgc)
print(" For XGboost : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred_cat)
print(" For Catboost : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred_lgc)
print(" For LightGBM  {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred_lda)
print(" For LDA: {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred_qda)
print(" For QDA: {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred_rf)
print(" For RF: {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
accuracy,precision,recall=confusion_matrix(y_test,pred_gb)
print(" For GB: {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
X_train.shape[1]

In [ ]:
X

# Main Results

#### LSTM

In [ ]:

#LSTM
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
input_dim = X_train.shape[1]  # Number of features
num_classes=2
model=Sequential()
model.add(Embedding(input_dim,50,mask_zero=True))
model.add(LSTM(20,dropout=0.4,return_sequences=True))
model.add(LSTM(20,dropout=0.5,return_sequences=False))
#model.add(LSTM(20,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
#model.add(LSTM(20,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=1000)

pred=model.predict(X_test)
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

accuracy_score(y_test,pred_lstm)

####BiLSTM

In [ ]:
%%time
#BiLSTM
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
input_dim = X_train.shape[1]  # Number of features
num_classes=2
model=Sequential()
model.add(Embedding(input_dim,50,mask_zero=True))
model.add(Bidirectional(LSTM(20,dropout=0.4, recurrent_dropout=0.4,return_sequences=True)))
model.add(Bidirectional(LSTM(20,dropout=0.5, recurrent_dropout=0.5,return_sequences=False)))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=1000)

pred=model.predict(X_test)
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

accuracy_score(y_test,pred_lstm)

#### Hybrid LSTM

In [ ]:
!pip uninstall keras
!pip install keras==2.4.0

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [ ]:
#%%time
#LSTM
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Flatten
from keras.layers import Dense, LSTM, Convolution1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Flatten
from keras.layers import Dense, LSTM, Convolution1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
input_dim = X_train.shape[1]  # Number of features

num_classes=2
model=Sequential()
model.add(Embedding(input_dim,50,mask_zero=True))
#model.add(Convolution1D(filters=128, kernel_size=3, padding='valid', activation='relu'))
model.add(Convolution1D(filters=128, kernel_size=3, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(4))
model.add(LSTM(20,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(20,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Flatten())
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=400)

pred=model.predict(X_test)
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

accuracy_score(y_test,pred_lstm)

In [ ]:
#%%time
#LSTM
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Flatten
from keras.layers import Dense, LSTM, Convolution1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Flatten
from keras.layers import Dense, LSTM, Convolution1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
input_dim = X_train.shape[1]  # Number of features

num_classes=2
model=Sequential()
model.add(Embedding(input_dim,50,mask_zero=True))
#model.add(Convolution1D(filters=128, kernel_size=3, padding='valid', activation='relu'))
model.add(Convolution1D(filters=128, kernel_size=3, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(4))
model.add(Bidirectional(LSTM(20,dropout=0.4, recurrent_dropout=0.4,return_sequences=True)))
model.add(Bidirectional(LSTM(20,dropout=0.5, recurrent_dropout=0.5,return_sequences=False)))
model.add(Flatten())
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=400)

pred=model.predict(X_test)
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

accuracy_score(y_test,pred_lstm)

In [ ]:
"""
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Flatten
from keras.layers import Dense, LSTM, Convolution1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional

QA_EMBED_SIZE = 64
DROPOUT_RATE = 0.3

model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(LSTM(QA_EMBED_SIZE, return_sequences=True, dropout=DROPOUT_RATE, recurrent_dropout=DROPOUT_RATE)))
model.add(Convolution1D(filters=128, kernel_size=3, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(4))
model.add(Flatten())

model.add(Dense(QA_EMBED_SIZE))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation("sigmoid"))

model.summary()
"""

https://www.kaggle.com/syedsaqlainhussain/sql-injection-dataset?select=sqliv2.csv

In [ ]:
from keras.models import load_model
import pickle

model.save('my_model_cnn.h5')
with open('vectorizer_cnn', 'wb') as fin:
    pickle.dump(vectorizer, fin)


In [ ]:

import keras
from keras.models import load_model
import pickle
import tensorflow as tf


mymodel = tf.keras.models.load_model('my_model_cnn.h5')
myvectorizer = pickle.load(open("vectorizer_cnn", 'rb'))




def clean_data(input_val):

    input_val=input_val.replace('\n', '')
    input_val=input_val.replace('%20', ' ')
    input_val=input_val.replace('=', ' = ')
    input_val=input_val.replace('((', ' (( ')
    input_val=input_val.replace('))', ' )) ')
    input_val=input_val.replace('(', ' ( ')
    input_val=input_val.replace(')', ' ) ')
    input_val=input_val.replace('1 ', 'numeric')
    input_val=input_val.replace(' 1', 'numeric')
    input_val=input_val.replace("'1 ", "'numeric ")
    input_val=input_val.replace(" 1'", " numeric'")
    input_val=input_val.replace('1,', 'numeric,')
    input_val=input_val.replace(" 2 ", " numeric ")
    input_val=input_val.replace(' 3 ', ' numeric ')
    input_val=input_val.replace(' 3--', ' numeric--')
    input_val=input_val.replace(" 4 ", ' numeric ')
    input_val=input_val.replace(" 5 ", ' numeric ')
    input_val=input_val.replace(' 6 ', ' numeric ')
    input_val=input_val.replace(" 7 ", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace('1234', ' numeric ')
    input_val=input_val.replace("22", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace(" 200 ", ' numeric ')
    input_val=input_val.replace("23 ", ' numeric ')
    input_val=input_val.replace('"1', '"numeric')
    input_val=input_val.replace('1"', '"numeric')
    input_val=input_val.replace("7659", 'numeric')
    input_val=input_val.replace(" 37 ", ' numeric ')
    input_val=input_val.replace(" 45 ", ' numeric ')

    return input_val








def predict_sqli_attack():
    
    repeat=True
    
    beautify=''
    for i in range(20):
        beautify+= "*"

    print(beautify) 
    input_val=input("Enter a sentence : ")


    
    if input_val== '0':
        repeat=False
    
    

    input_val=clean_data(input_val)
    input_val=[input_val]



    input_val=myvectorizer.transform(input_val).toarray()
    
   # input_val.shape=(1,64,64,1)

    result=mymodel.predict(input_val)


#     print(beautify
    
    
    if repeat == True:
        
        if result>0.5:
            print("ALERT!!!! SQL injection Detected")


        elif result<=0.5:
            print("It is normal")
            
        print(beautify)
            
        predict_sqli_attack()
            
    elif repeat == False:
        print( " Closing detection ")

 



In [ ]:
# Uncomment the function call below and enter the strings to detect the SQL injection attack

# predict_sqli_attack()